In [7]:
import ee
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from mpetools import get_info_islands
import geemap

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [41]:
island = 'Banaba'
country = 'Kiribati'
island_info = get_info_islands.retrieve_info_island(island, country)


-------------------------------------------------------------------
RETRIEVING ISLAND INFORMATION
Island: Banaba, Kiribati
-------------------------------------------------------------------

~ The following information is available: ~

general_info
               island
               country
spatial_reference
               latitude
               longitude
               polygon
image_collection_dict
               description
               S2
               L7
               L8
               L9


In [42]:
aoi = island_info['spatial_reference']['polygon']

# Filter the Sentinel-1 collection
sentinel1_collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                        .filterBounds(aoi)
                        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
                        .select(['VV', 'VH'])
                        .map(lambda image: image.clip(aoi)))

# Get the first image from the collection
sentinel1_image = sentinel1_collection.first()

In [43]:
ee.Date(sentinel1_image.getInfo()['properties']['system:time_start']).format('YYYY-MM-dd').getInfo()

TypeError: 'NoneType' object is not subscriptable

In [33]:
m = geemap.Map(center=[island_info['spatial_reference']['latitude'], island_info['spatial_reference']['longitude']], zoom=12)
m.addLayer(sentinel1_image, vis_params={'min': -25, 'max': 0})
m

Map(center=[0.6209706500000001, 73.46182384568081], controls=(WidgetControl(options=['position', 'transparent_…

In [48]:
# Define the Sentinel-1 VV image collection
imgVV = (ee.ImageCollection('COPERNICUS/S1_GRD')
         .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
         .filter(ee.Filter.eq('instrumentMode', 'IW'))
         .select('VV')
         .map(lambda image: image.updateMask(image.lt(-30.0).Not())))

# Filter the VV images by pass direction
desc = imgVV.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
asc = imgVV.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

# Define the date ranges
spring = ee.Filter.date('2022-03-01', '2022-04-20')
lateSpring = ee.Filter.date('2022-04-21', '2022-06-10')
summer = ee.Filter.date('2022-06-11', '2022-08-31')

# Compute the multi-temporal means for DESCENDING pass
descChange = (ee.Image.cat([
    desc.filter(spring).mean(),
    desc.filter(lateSpring).mean(),
    desc.filter(summer).mean()
]))

# Compute the multi-temporal means for ASCENDING pass
ascChange = (ee.Image.cat([
    asc.filter(spring).mean(),
    asc.filter(lateSpring).mean(),
    asc.filter(summer).mean()
]))

Map = geemap.Map(center=[5.2013, 47.3277], zoom=12)
# Add layers to the map
Map.addLayer(ascChange, {'min': -25, 'max': 5}, 'Multi-T Mean ASC', True)
Map.addLayer(descChange, {'min': -25, 'max': 5}, 'Multi-T Mean DESC', True)
Map

Map(center=[5.2013, 47.3277], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…